In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy.stats as stats
from GmGM import GmGM, Dataset

In [2]:
def svd_rank_one_update(U, S, V, a):
    """
    Computes SVD of USV^T + a1^T via rank one update.
    """
    p, r = U.shape
    q, _ = V.shape

    # Make sure this constitutes a valid SVD
    assert S.shape == (r,)
    assert a.shape == (p,)
    assert U.shape == (p, r)
    assert V.shape == (q, r)

    # Orthogonal projection vectors
    m = U.T @ a
    P = a - U @ m
    R_a = np.linalg.norm(P)
    P /= R_a

    n = V.sum(axis=0)
    Q = 1 - V @ n
    R_b = np.linalg.norm(Q)
    Q /= R_b


    # Create the K that should be eigendecomped
    K1 = np.zeros((r+1, r+1))
    np.fill_diagonal(K1[:r, :r], S)
    K2 = (
        np.concatenate((m, np.array([R_a]))).reshape(-1, 1)
        @ np.concatenate((n, np.array([R_b]))).reshape(1, -1)
    )
    K = K1 + K2

    # Inner eigendecomp
    Up, Sf, VpT = np.linalg.svd(K)
    Vp = VpT.T

    # Results
    Uf = np.hstack((U, P.reshape(-1, 1))) @ Up
    Vf = np.hstack((V, Q.reshape(-1, 1))) @ Vp
    
    return Uf, Sf, Vf

In [3]:
def sparse_normal_map(X: sparse.sparray) -> tuple[sparse.sparray, np.ndarray]:
    """
    Given a sparse matrix X (p by q), maps it to a normal distribution.
    To preserve sparsity, we return the output expressed as a sum:
    A + zeromaps @ np.ones(q)^T
    where A has the same sparsity pattern as X, and zeromaps is a p-vector
    containing the value (per-row) that zero was mapped to by the transformation.

    This enables us to operate on a sparse matrix A, and use zeromaps later for
    rank-one updates of those operations.  This helps avoid the need to densify.
    """
    p, q = X.shape
    A = X.copy()

    zeromaps = np.zeros(p)
    for i in range(p):
        Y = X[[i], :].toarray()
        cur = stats.rankdata(Y, axis=1)
        cur = stats.norm.ppf(cur / (q+1))
        if 0 in Y:
            rank = (Y < 0).sum() + 1
            zeromaps[i] = stats.norm.ppf(rank / (q+1))
        else:
            zeromaps[i] = 0
        cur -= zeromaps[i]

        # Looks complicated, but is needed because:
        # X[[i], :] is not a view, and hence X[[i], :][Y != 0]
        # is not assignable to!  (It sets values in a copy that gets
        # immediately deleted)
        A[np.ix_([i], (Y != 0).flatten())] = cur[Y != 0]

    return A, zeromaps

In [4]:
# Parameters
p, q, r = 100, 50, 40
s = 0.1

# Generate data
X = sparse.csr_array(sparse.random(p, q, density=s, format='csr'))

# # Sparsity-preserving nonparanormal skeptic
# zeromaps = np.zeros(p)
# for i in range(p):
#     Y = X[[i], :].toarray()
#     cur = stats.rankdata(Y, axis=1)
#     cur = stats.norm.ppf(cur / (q+1))
#     if 0 in Y:
#         rank = (Y < 0).sum() + 1
#         zeromaps[i] = stats.norm.ppf(rank / (q+1))
#     else:
#         zeromaps[i] = 0
#     cur -= zeromaps[i]

#     # Looks complicated, but is needed because:
#     # X[[i], :] is not a view, and hence X[[i], :][Y != 0]
#     # is not assignable to!  (It sets values in a copy that gets
#     # immediately deleted)
#     X[np.ix_([i], (Y != 0).flatten())] = cur[Y != 0]

X, zeromaps = sparse_normal_map(X)

# Check the conversion has gone well
X_ = stats.rankdata(X.toarray(), axis=1, method='min')
X_ = stats.norm.ppf(X_ / (q+1))
print(((X - X_ + zeromaps.reshape(-1, 1)) < 1e-10).all())


# Compute SVD
U, S, VT = sparse.linalg.svds(X, k=r)
V = VT.T

# Only compare to low-rank matrix:
X = U @ np.diag(S) @ V.T

# Rank one update
a = zeromaps
Uf, Sf, Vf = svd_rank_one_update(U, S, V, a)
Xf = Uf @ np.diag(Sf) @ Vf.T

# Ground truth
Xt = X + a.reshape(-1, 1)
Ut, St, VtT = np.linalg.svd(Xt)
St = St[:r+1]
Ut = Ut[:, :r+1]
Vt = VtT.T[:, :r+1]

# Results
Sdiff = np.linalg.norm(St.flatten() - Sf.flatten())
Udiff = np.linalg.norm(abs(Ut.T@Uf) - np.eye(r+1))
Vdiff = np.linalg.norm(abs(Vt.T@Vf) - np.eye(r+1))
print(Sdiff)
print(Udiff)
print(Vdiff)

True
1.329557779806941e-13
2.846674376856255e-13
2.863427600972299e-13


In [5]:
?Dataset

Init signature:
Dataset(
    *,
    dataset: 'dict[Modality, DataTensor]',
    structure: 'dict[Modality, tuple[Axis]]',
    batch_axes: 'Optional[set[Axis]]' = None,
)
Docstring:      <no docstring>
File:           ~/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/dataset.py
Type:           type
Subclasses:     

In [72]:
p, q, r = 400, 1000, 50
s = 0.5

# Generate data
raw = sparse.csr_array(sparse.random(p, q, density=s, format='csr'))
raw2 = raw.toarray()
raw3 = raw.copy().toarray()
X = Dataset(
    dataset={"a": raw},
    structure={"a": ("b", "c")},
)
Y = Dataset(
    dataset={"a": raw2},
    structure={"a": ("b", "c")},
)

a = GmGM(X, n_comps=r, verbose=False, to_keep=1, use_nonparanormal_skeptic=True).precision_matrices["b"]
b = GmGM(Y, n_comps=r, verbose=False, to_keep=1, use_nonparanormal_skeptic=True).precision_matrices["b"]
print(((a != 0).toarray() == (b != 0).toarray()).sum() / (p*p))

/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/dataset.py:163: UserWarning: Trying to set a's tensor of type <class 'scipy.sparse._csr.csr_array'> to read-only, but this dataset has no `flags` attribute.  Making a copy instead.
  warnings.warn(
/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/core/core.py:110: UserWarning: `nonparanormal_evec_backend` unspecified, defaulting to `COCA`
  warnings.warn("`nonparanormal_evec_backend` unspecified, defaulting to `COCA`")
/Users/baileyandrew/mambaforge/envs/GmGM-python-accelerate/lib/python3.9/site-packages/GmGM/core/core.py:60: UserWarning: `nonparanormal_evec_backend` unspecified, defaulting to `COCA`
  warnings.warn("`nonparanormal_evec_backend` unspecified, defaulting to `COCA`")


0.9966375
